# D&D statistics and functions

In [2]:
import random
import numpy as np
import pandas as pd
from dataclasses import dataclass

The functions below will be exemplified with a party comprising
- Diane
- Peter
- Robin

## Individual and team initiatives

Calculating individual and team initiatives over 10k iterations, taking into account each character's initiative bonus and whether the character rolls initiative at advantage or at single roll.

In [6]:
def individual_initiative(name, init_bonus, roll_type='single'):
    total_initiative = 0
    iterations = 10000

    for _ in range(iterations + 1):
        if roll_type == 'adv':
            roll_result = max([random.randint(1, 20),
                               random.randint(1, 20)])
        else:
            roll_result = random.randint(1, 20)
        total_initiative += roll_result + init_bonus

# minimum initiative defined as a natural 1 on the die + initiative bonus
# maximum initiative defined as a natural 20 on the die + initiative bonus
# average initiative defined as arithmetic average of initiative over iterations
    min_initiative = 1 + init_bonus
    max_initiative = 20 + init_bonus
    avg_initiative = total_initiative / iterations

# printing the results of the function rounded to the nearest whole number
    print(f"{name}'s average initiative is {avg_initiative:.0f}.")
    print(f"{name}'s minimum initiative is {min_initiative:.0f}.")
    print(f"{name}'s maximum initiative is {max_initiative:.0f}.")

In [7]:
individual_initiative(name= 'Diane',
                      init_bonus= 4,
                      roll_type= 'adv')

Diane's average initiative is 18.
Diane's minimum initiative is 5.
Diane's maximum initiative is 24.


In [8]:
def team_initiative(team_data):

    '''
for each character in the team, 
1. initiative bonus and roll type are extracted
2. 10k iterations are calculated with either normal or advantaged rolls
3. min, max and avg are determined for the character
4. the list is appended with the results for each character
5. a dataframe is returned with the team's initiatives
    '''
    # storing the initiatives calculated in the loop in a list
    initiatives = []

    for name, details in team_data.items():
        init_bonus = details['initiative_bonus']
        roll_type = details['roll_type']

        total_initiative = 0
        iterations = 10000
    
        for _ in range(iterations + 1):
            if roll_type == 'adv':
                roll_result = max([random.randint(1, 20),
                                   random.randint(1, 20)])
            else:
                roll_result = random.randint(1, 20)
            total_initiative += roll_result + init_bonus

        min_initiative = 1 + init_bonus
        max_initiative = 20 + init_bonus
        avg_initiative = total_initiative / iterations

        initiatives.append({
        'character_name': name,
        'min_initiative': min_initiative,
        'avg_initiative': np.round(avg_initiative, 1),
        'max_initiative': max_initiative
        })
    
    team_initiatives = pd.DataFrame(initiatives)
    return team_initiatives

In [9]:
# sample team data to calculate initiative
party_initiatives = {
    'Diane': {'initiative_bonus': 4, 'roll_type': 'adv'},
    'Peter': {'initiative_bonus': 2, 'roll_type': 'single'},
    'Robin': {'initiative_bonus': 8, 'roll_type': 'single'}
}

team_initiative(party_initiatives)

,character_name,min_initiative,avg_initiative,max_initiative
0,Diane,5,17.8,24
1,Peter,3,12.5,22
2,Robin,9,18.6,28


## AC and probability to hit

In [11]:
def calculate_hit_probabilities(characters_dict, target_value):
    '''
    Calculate hit probabilities for multiple characters against a target AC 
    whether the roll is at disadvantage, advantage or normal.
    
    Arguments:
        - characters_dict: Dictionary of {character_name: attack_bonus}
        - target_value: AC being targeted
    
    Returns:
        pandas.DataFrame with probabilities for each character for each type of roll.
    '''
    
    def calculate_single(bonus, dc):
        required_roll = dc - bonus
        success_range = 21 - required_roll
        success_range = max(0, min(success_range, 20))
        return (success_range / 20) * 100

    def calculate_advantage(bonus, dc):
        base_prob = calculate_single(bonus, dc) / 100
        return (1 - (1 - base_prob)**2) * 100

    def calculate_disadvantage(bonus, dc):
        base_prob = calculate_single(bonus, dc) / 100
        return (base_prob**2) * 100

    results = []
    for name, bonus in characters_dict.items():
        results.append({
            'name': name,
            'hit % at disadvantage': round(calculate_disadvantage(bonus, target_value)),
            'hit % at single roll': round(calculate_single(bonus, target_value)),
            'hit % at advantage': round(calculate_advantage(bonus, target_value))
        })

    return pd.DataFrame(results)[['name', 
                                  'hit % at disadvantage',
                                  'hit % at single roll', 
                                  'hit % at advantage']]

In [12]:
hit_modifiers = {
    'Diane': 12,
    'Peter': 11,
    'Robin': 10
}

calculate_hit_probabilities(characters_dict= hit_modifiers,
                            target_value= 19)

,name,hit % at disadvantage,hit % at single roll,hit % at advantage
0,Diane,49,70,91
1,Peter,42,65,88
2,Robin,36,60,84


## Spell or attack damage calculations

To calculate total damage per round for either spellcasters or martial characters, we need to
1. parse the string representing damage (e.g., `2d6+3` or `4d8`)
2. coding damage profiles for each possible attack with inputs the results from the previous parse function (e.g., `2d6+3 slashing damage plus 1d6 necrotic`)
3. coding the attack sequence applying the damage profile for each possible attack per turn for the character

In [15]:
def parse_damage(damage_str):
    if '+' in damage_str:
        dice, bonus = damage_str.split('+')
        bonus = int(bonus)
    else:
        dice = damage_str
        bonus = 0

        # split damage to get # dice & # sides per die
    dice_count, dice_sides = map(int, dice.split('d'))

    return dice_count, dice_sides, bonus

@dataclass
class DamageProfile:
    min: float
    avg: float
    max: float

def create_damage_profile(damage_str: str,
                          min_roll: int = 1) -> DamageProfile:
        dice_count, dice_sides, bonus = parse_damage(damage_str)
        adj_avg = (min_roll + dice_sides) / 2
        return DamageProfile(
            min = dice_count * min_roll + bonus,
            avg = dice_count * adj_avg + bonus,
            max = dice_count * dice_sides + bonus
        )

def calculate_total_damage(damage_profiles: list[DamageProfile], 
                          max_attacks: int) -> pd.DataFrame:
    '''
1. taking input from the damage profiles previously defined
2. aggregating the minimum, average & maximum damages of all profiles
3. defining an attack sequence (# of possible hits in a turn) and applying the 
    aggregated damage profiles cumulatively.
    '''
    # Aggregate base damage
    total_min = sum(p.min for p in damage_profiles)
    total_avg = sum(p.avg for p in damage_profiles)
    total_max = sum(p.max for p in damage_profiles)
    
    # Build attack sequence
    results = []
    for hits in range(1, max_attacks + 1):
        results.append((
            hits,
            total_min * hits,
            total_avg * hits,
            total_max * hits
        ))
    
    return pd.DataFrame(results, 
                       columns=['Hit#', 'Min Damage', 
                               'Avg Damage', 'Max Damage'])

To test this function, let's analyze Robin's glaive that has a slashing damage profile of `1d10+4` and an additional necrotic damage profile of `1d4`. Robin attacks three times per turn with extra attack.

In [17]:
Robin_slashing = create_damage_profile('1d10+4')
Robin_necrotic = create_damage_profile('1d6')

calculate_total_damage([Robin_slashing, Robin_necrotic], 3)

,Hit#,Min Damage,Avg Damage,Max Damage
0,1,6,13.0,20
1,2,12,26.0,40
2,3,18,39.0,60


Diane is a spellcaster with access to Meteor Swarm, which deals `20d6` fire damage and `20d6` bludgeoning damage with a dexterity save allowing for half damage if the save is successful.

The next function calculates the half and full damage of spells like Meteor Swarm.

In [19]:
def calculate_half_full_damage(damage_profiles: list[DamageProfile]) -> pd.DataFrame:
    total_min = sum(p.min for p in damage_profiles)
    total_avg = sum(p.avg for p in damage_profiles)
    total_max = sum(p.max for p in damage_profiles)

    data = {
        '': ['Failed Save', 'Successful Save'],
        'Min Damage': [total_min, total_min / 2],
        'Avg Damage': [total_avg, total_avg / 2],
        'Max Damage': [total_max, total_max / 2]
    }

    return pd.DataFrame(data)

In [20]:
fire= create_damage_profile('20d6')
bludgeoning= fire # bludgeoning profile is identical to fire profile

calculate_half_full_damage([fire, bludgeoning])

,,Min Damage,Avg Damage,Max Damage
0,Failed Save,40.0,140.0,240.0
1,Successful Save,20.0,70.0,120.0


## Skill and save success probabilities

The next function is to analyze the probability for each character's skills to succeed at different common DCs to identify each character's best skills.

1. Defining base success probabilities for single roll, advantaged roll and disadvantaged roll.
2. Defining DC values of interest as a numpy array to allow for later vectorization.
3. Taking in dictionaries of character's skills or saves, calculating vectorized probability for each skill to be successful at all DCs.

In [23]:
def single_roll(modifier, dc):
        base_prob = (21 - np.maximum(dc - modifier, 0)) / 20
        return np.clip(base_prob, 0.05, 0.95)
    
def adv_roll(modifier, dc):
    single_prob = single_roll(modifier, dc)
    return np.clip(1 - (1 - single_prob) ** 2, 0.1, 0.9975)
    
def dis_roll(modifier, dc):
    single_prob = single_roll(modifier, dc)
    return np.clip(single_prob ** 2, 0.025, 0.9025)

dc_values = np.array([10, 12, 15, 17, 20, 22, 25, 30])

def skill_save_success(character_data): 
    results = {}
        
    for skill_save, details in character_data.items():
        modifier = details['modifier']
        
        if details['roll_type'] == 'adv':
            probs = adv_roll(modifier, dc_values)
        elif details['roll_type'] == 'dis':
            probs = dis_roll(modifier, dc_values)
        else:
            probs = single_roll(modifier, dc_values)
        
        results[skill_save] = probs * 100
        
    return pd.DataFrame(results, index= dc_values).T.round(2)

Peter's skills and saves will be analyzed as the party's rogue with two tables
- the skills/saves are as rows
- the various DCs of interest are as columns
- the probabilities are as values, formatted numerically to allow for further analysis

In [25]:
Peter_skills = {
    'Acrobatics': {'modifier': 9, 'roll_type': 'single'},
    'Animal Handling': {'modifier': 3, 'roll_type': 'single'},
    'Arcana': {'modifier': 5, 'roll_type': 'single'},
    'Athletics': {'modifier': 1, 'roll_type': 'single'},
    'Deception': {'modifier': 5, 'roll_type': 'single'},
    'History': {'modifier': 2, 'roll_type': 'single'},
    'Insight': {'modifier': 3, 'roll_type': 'single'},
    'Intimidation': {'modifier': 10, 'roll_type': 'single'},
    'Investigation': {'modifier': 5, 'roll_type': 'single'},
    'Medicine': {'modifier': 3, 'roll_type': 'single'},
    'Nature': {'modifier': 2, 'roll_type': 'single'},
    'Perception': {'modifier': 6, 'roll_type': 'single'},
    'Performance': {'modifier': 5, 'roll_type': 'single'},
    'Persuasion': {'modifier': 5, 'roll_type': 'single'},
    'Religion': {'modifier': 2, 'roll_type': 'single'},
    'Sleight of Hand': {'modifier': 14, 'roll_type': 'single'},
    'Stealth': {'modifier': 14, 'roll_type': 'adv'},
    'Survival': {'modifier': 3, 'roll_type': 'single'}
}

Peter_saves = {
    'Strength': {'modifier': -1, 'roll_type': 'adv'},
    'Dexterity': {'modifier': 8, 'roll_type': 'single'},
    'Constitution': {'modifier': 1, 'roll_type': 'single'},
    'Intelligence': {'modifier': 4, 'roll_type': 'single'},
    'Wisdom': {'modifier': 1, 'roll_type': 'single'},
    'Charisma': {'modifier': 4, 'roll_type': 'single'}
}

In [26]:
skill_save_success(Peter_skills)

,10,12,15,17,20,22,25,30
Acrobatics,95.00,90.00,75.00,65.0,50.00,40.00,25.0,5.00
Animal Handling,70.00,60.00,45.00,35.0,20.00,10.00,5.0,5.00
Arcana,80.00,70.00,55.00,45.0,30.00,20.00,5.0,5.00
Athletics,60.00,50.00,35.00,25.0,10.00,5.00,5.0,5.00
Deception,80.00,70.00,55.00,45.0,30.00,20.00,5.0,5.00
History,65.00,55.00,40.00,30.0,15.00,5.00,5.0,5.00
Insight,70.00,60.00,45.00,35.0,20.00,10.00,5.0,5.00
Intimidation,95.00,95.00,80.00,70.0,55.00,45.00,30.0,5.00
Investigation,80.00,70.00,55.00,45.0,30.00,20.00,5.0,5.00
Medicine,70.00,60.00,45.00,35.0,20.00,10.00,5.0,5.00


In [27]:
skill_save_success(Peter_saves)

,10,12,15,17,20,22,25,30
Strength,75.0,64.0,43.75,27.75,10.0,10.0,10.0,10.0
Dexterity,95.0,85.0,70.00,60.00,45.0,35.0,20.0,5.0
Constitution,60.0,50.0,35.00,25.00,10.0,5.0,5.0,5.0
Intelligence,75.0,65.0,50.00,40.00,25.0,15.0,5.0,5.0
Wisdom,60.0,50.0,35.00,25.00,10.0,5.0,5.0,5.0
Charisma,75.0,65.0,50.00,40.00,25.0,15.0,5.0,5.0
